Mark Oulitin 208283291
Itay Cohen 211896261

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

In [2]:
loans = pd.read_csv('input/should-this-loan-be-approved-or-denied/SBAnational.csv')
loans.head()

c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [3]:
loans.shape

(899164, 27)

Data Preparation

In [4]:
# we want to make a binary value stating if the loan was defaulted or not
loans['Defaulted'] = [1 if app == 'CHGOFF' else 0 for app in loans.MIS_Status.values]
loans = loans.drop(columns=['MIS_Status'])

# we only take the loans created before 2007
loans['ApprovalFY'] = loans['ApprovalFY'].replace({'A':'','B':''}, regex = True).astype(int)
loans.drop(loans[loans['ApprovalFY']>2007].index, axis = 0, inplace = True)

loans = loans.drop(columns=['ApprovalFY','ApprovalDate','DisbursementDate','ChgOffDate','LoanNr_ChkDgt','Name','Zip', 'City'])

In [5]:
loans.head()

,State,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,ChgOffPrinGr,GrAppv,SBA_Appv,Defaulted
0,IN,FIFTH THIRD BANK,OH,451120,84,4,2.0,0,0,1,0,N,Y,"$60,000.00",$0.00,$0.00,"$60,000.00","$48,000.00",0
1,IN,1ST SOURCE BANK,IN,722410,60,2,2.0,0,0,1,0,N,Y,"$40,000.00",$0.00,$0.00,"$40,000.00","$32,000.00",0
2,IN,GRANT COUNTY STATE BANK,IN,621210,180,7,1.0,0,0,1,0,N,N,"$287,000.00",$0.00,$0.00,"$287,000.00","$215,250.00",0
3,OK,1ST NATL BK & TR CO OF BROKEN,OK,0,60,2,1.0,0,0,1,0,N,Y,"$35,000.00",$0.00,$0.00,"$35,000.00","$28,000.00",0
4,FL,FLORIDA BUS. DEVEL CORP,FL,0,240,14,1.0,7,7,1,0,N,N,"$229,000.00",$0.00,$0.00,"$229,000.00","$229,000.00",0


In [6]:
loans.shape

(802319, 19)

In [7]:
def amount_to_float(number):
    num = number.replace("$", "")
    num = num.replace(",","")
    num = num.replace(" ","")
    return float(num)

#instead of dealing with money as a string we will deal with it as a number
loans['BalanceGross'] = loans['BalanceGross'].apply(lambda x: amount_to_float(x))
loans['DisbursementGross'] = loans['DisbursementGross'].apply(lambda x: amount_to_float(x))
loans['ChgOffPrinGr'] = loans['ChgOffPrinGr'].apply(lambda x: amount_to_float(x))
loans['GrAppv'] = loans['GrAppv'].apply(lambda x: amount_to_float(x))
loans['SBA_Appv'] = loans['SBA_Appv'].apply(lambda x: amount_to_float(x))

In [8]:
# we replace it to be a binary where 1 to be a new business and 0 an old business
loans['NewExist'] = loans['NewExist'].replace(1,0)
loans['NewExist'] = loans['NewExist'].replace(2,1)

In [9]:
# we replace them both to be binary instead of Y and N
loans['LowDoc'] = loans['LowDoc'].replace({'Y':'1', 'N':'0'}, regex=True)
valid = ['1', '0']
loans = loans.loc[loans['LowDoc'].isin(valid)]
loans['LowDoc'] = loans['LowDoc'].astype(int)
loans['RevLineCr'] = loans['RevLineCr'].replace({'Y':'1', 'N':'0'}, regex=True)
loans = loans.loc[loans['RevLineCr'].isin(valid)]
loans['RevLineCr'] = loans['RevLineCr'].astype(int)

In [10]:
# we make franchise a binary value where 1 is true and 0 is false
loans['FranchiseCode'] = loans['FranchiseCode'].replace(1,0)
loans['FranchiseCode'] = np.where((loans.FranchiseCode != 0),1,loans.FranchiseCode)
loans.rename(columns={"FranchiseCode":"Franchise"},inplace=True)

In [11]:
loans.shape

(783512, 19)

In [12]:
loans = pd.get_dummies(loans,columns=['BankState','State'],prefix=['BankState','State'])

In [13]:
#only take the first 2 digits from the NAICS values
def first_n_digits(number, n):
    return int(str(number)[:n])
loans['NAICS'] = loans['NAICS'].apply(lambda x: first_n_digits(x,2))

In [14]:
loans.head()

,Bank,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,Franchise,UrbanRural,RevLineCr,...,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY
0,FIFTH THIRD BANK,45,84,4,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1ST SOURCE BANK,72,60,2,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,GRANT COUNTY STATE BANK,62,180,7,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1ST NATL BK & TR CO OF BROKEN,0,60,2,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,FLORIDA BUS. DEVEL CORP,0,240,14,0.0,7,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
loans['NAICS'].unique()

array([45, 72, 62,  0, 33, 81, 23, 44, 42, 61, 53, 54, 31, 51, 71, 52, 21,
       32, 56, 48, 11, 92, 22, 49, 55], dtype=int64)

In [16]:
loans.dtypes

Bank         object
NAICS         int64
Term          int64
NoEmp         int64
NewExist    float64
             ...   
State_VT      uint8
State_WA      uint8
State_WI      uint8
State_WV      uint8
State_WY      uint8
Length: 123, dtype: object

Modeling

In [17]:
y = loans['Defaulted']
X = loans.drop(columns=['Defaulted'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)